# < Phase 4: 도메인 특화 피처 생성 >
- 선수별 패스 스타일 (4개)
- 팀별 전술 성향 (2개)  
- 경기 흐름 (3개)
**총 9개 피처 추가**

## 1. 환경설정

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [37]:
# 한글 폰트 설정 (Windows)
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

# 경로 설정
DATA_DIR = Path('../data/raw')
PROCESSED_DIR = Path('../data/processed')

In [38]:
# 데이터 파일 확인 및 크기 출력
print("📂 사용 가능한 데이터 파일:")
for file in DATA_DIR.glob('*.csv'):
    size = file.stat().st_size / 1024 / 1024  # MB
    print(f"  - {file.name}: {size:.2f} MB")

📂 사용 가능한 데이터 파일:
  - match_info.csv: 0.03 MB
  - sample_submission.csv: 0.05 MB
  - test.csv: 0.10 MB
  - train.csv: 41.54 MB


## 2. Data loading

In [39]:
print("=" * 50)
print("Phase 4: 도메인 특화 피처 생성")
print("=" * 50)

# 데이터 불러오기
# 기존 Phase 3 데이터
df_v3 = pd.read_csv(PROCESSED_DIR / 'train_final_passes_v3.csv')
print(f"\n✅ Phase 3 데이터 로드: {df_v3.shape}")

# 원본 train.csv (전체 액션)
train_full = pd.read_csv(DATA_DIR / 'train.csv')
print(f"✅ 원본 train 로드: {train_full.shape}")

# 경기 정보
match_info = pd.read_csv(DATA_DIR / 'match_info.csv')
print(f"✅ match_info 로드: {match_info.shape}")

Phase 4: 도메인 특화 피처 생성

✅ Phase 3 데이터 로드: (15435, 43)
✅ 원본 train 로드: (356721, 15)
✅ match_info 로드: (228, 17)


## 3. 선수/팀 통계 계산 (Data Leakage 방지)

In [40]:
print("\n" + "=" * 50)
print("선수/팀 통계 계산 (과거 데이터만 사용)")
print("=" * 50)

# Pass 액션만 필터링
passes = train_full[train_full['type_name'] == 'Pass'].copy()
print(f"\n📊 전체 Pass 액션: {len(passes):,}개")

# 패스 성공 여부 (result_name이 'Successful'이면 성공)
passes['is_success'] = passes['result_name'] == 'Successful'

# 패스 거리 계산
passes['pass_distance'] = np.sqrt(
    (passes['end_x'] - passes['start_x'])**2 + 
    (passes['end_y'] - passes['start_y'])**2
)

# 전진 패스 여부 (홈/원정 고려)
passes['forward_distance'] = np.where(
    passes['is_home'],
    passes['end_x'] - passes['start_x'],  # 홈: 오른쪽이 공격
    passes['start_x'] - passes['end_x']   # 원정: 왼쪽이 공격
)
passes['is_forward'] = passes['forward_distance'] > 0

# 측면 vs 중앙 (y축 기준)
passes['is_wide'] = (passes['start_y'] < 20) | (passes['start_y'] > 48)

print("✅ 패스 통계 변수 생성 완료")


선수/팀 통계 계산 (과거 데이터만 사용)

📊 전체 Pass 액션: 178,582개
✅ 패스 통계 변수 생성 완료


## 4. 경기별 누적 통계 계산

In [41]:
def calculate_cumulative_stats(df, group_col, stat_cols):
    """
    각 경기에 대해 이전 경기들의 통계를 계산
    
    Parameters:
    - df: 데이터프레임
    - group_col: 그룹 컬럼 (player_id 또는 team_id)
    - stat_cols: 집계할 통계 컬럼들
    
    Returns:
    - 통계 딕셔너리 {game_id: {group_id: {stat: value}}}
    """
    # game_id로 정렬
    df_sorted = df.sort_values('game_id').copy()
    
    # 각 경기의 통계 저장
    stats_by_game = {}
    
    # 누적 통계
    cumulative_stats = {}
    
    unique_games = sorted(df_sorted['game_id'].unique())
    
    for i, game_id in enumerate(unique_games):
        # 현재 경기 이전 데이터
        if i == 0:
            # 첫 경기는 통계 없음 (전체 평균 사용)
            stats_by_game[game_id] = None
        else:
            prev_data = df_sorted[df_sorted['game_id'].isin(unique_games[:i])]
            
            # 그룹별 통계 계산
            group_stats = prev_data.groupby(group_col).agg(stat_cols).to_dict('index')
            stats_by_game[game_id] = group_stats
    
    return stats_by_game

In [42]:
print("\n📈 선수별 누적 통계 계산 중...")

# 선수별 통계 정의
player_stat_cols = {
    'pass_distance': 'mean',
    'is_forward': 'mean',
    'is_success': 'mean',
    'player_id': 'count'  # 패스 횟수
}

player_stats_by_game = calculate_cumulative_stats(
    passes, 
    'player_id', 
    player_stat_cols
)

print("✅ 선수 통계 완료")


📈 선수별 누적 통계 계산 중...
✅ 선수 통계 완료


In [43]:
print("\n📈 팀별 누적 통계 계산 중...")

# 팀별 통계 정의
team_stat_cols = {
    'pass_distance': 'mean',
    'is_wide': 'mean'  # 측면 선호도
}

team_stats_by_game = calculate_cumulative_stats(
    passes,
    'team_id',
    team_stat_cols
)

print("✅ 팀 통계 완료")


📈 팀별 누적 통계 계산 중...
✅ 팀 통계 완료


## 5. 전체 평균 계산 (신인 대비용)

In [44]:
global_player_avg = {
    'pass_distance': passes['pass_distance'].mean(),
    'is_forward': passes['is_forward'].mean(),
    'is_success': passes['is_success'].mean(),
    'player_id': 10  # 기본값
}

global_team_avg = {
    'pass_distance': passes['pass_distance'].mean(),
    'is_wide': passes['is_wide'].mean()
}

print(f"\n🌍 전체 평균값:")
print(f"  - 평균 패스 거리: {global_player_avg['pass_distance']:.2f}m")
print(f"  - 전진 패스 비율: {global_player_avg['is_forward']:.1%}")
print(f"  - 패스 성공률: {global_player_avg['is_success']:.1%}")
print(f"  - 측면 선호도: {global_team_avg['is_wide']:.1%}")


🌍 전체 평균값:
  - 평균 패스 거리: 16.84m
  - 전진 패스 비율: 49.9%
  - 패스 성공률: 86.3%
  - 측면 선호도: 60.5%


## 6. Phase 3 데이터에 피처 추가

In [45]:
print("\n" + "=" * 50)
print("Phase 4 피처 추가 중...")
print("=" * 50)

# game_episode 형식 확인
print(f"\n📋 game_episode 샘플:")
print(df_v3['game_episode'].head(3).tolist())


Phase 4 피처 추가 중...

📋 game_episode 샘플:
['126283_1', '126283_10', '126283_11']


In [46]:
# train.csv에서 최종 패스의 player_id, team_id, is_home 가져오기
# 각 episode의 마지막 Pass만 필요
final_passes_info = train_full[train_full['type_name'] == 'Pass'].copy()

# episode별 마지막 액션 (time_seconds 기준)
final_passes_info = final_passes_info.sort_values('time_seconds')
final_passes_info = final_passes_info.groupby('game_episode').last().reset_index()

# 필요한 컬럼만 선택
final_passes_info = final_passes_info[['game_episode', 'game_id', 'team_id', 'player_id', 'is_home']]

print(f"\n✅ 최종 패스 정보 추출: {len(final_passes_info):,}개")

# Phase 3 데이터와 merge
df_v3 = df_v3.merge(final_passes_info, on='game_episode', how='left', suffixes=('', '_final'))

# game_id가 이미 있으면 제거
if 'game_id_final' in df_v3.columns:
    df_v3 = df_v3.drop(columns=['game_id_final'])

print(f"\n✅ team_id, player_id, is_home 추가 완료")


✅ 최종 패스 정보 추출: 15,435개

✅ team_id, player_id, is_home 추가 완료


### 6.1 선수 통계 피처

In [47]:
def get_player_stats(row):
    """해당 경기의 선수 통계 가져오기"""
    game_id = row['game_id']
    player_id = row['player_id']
    
    stats = player_stats_by_game.get(game_id)
    
    if stats is None or player_id not in stats:
        # 통계 없으면 전체 평균
        return pd.Series({
            'player_avg_pass_distance': global_player_avg['pass_distance'],
            'player_forward_ratio': global_player_avg['is_forward'],
            'player_success_rate': global_player_avg['is_success'],
            'player_pass_count': global_player_avg['player_id']
        })
    
    player_stat = stats[player_id]
    return pd.Series({
        'player_avg_pass_distance': player_stat.get(('pass_distance', 'mean'), global_player_avg['pass_distance']),
        'player_forward_ratio': player_stat.get(('is_forward', 'mean'), global_player_avg['is_forward']),
        'player_success_rate': player_stat.get(('is_success', 'mean'), global_player_avg['is_success']),
        'player_pass_count': player_stat.get(('player_id', 'count'), global_player_avg['player_id'])
    })

print("\n🏃 선수 통계 피처 생성 중...")
player_features = df_v3.apply(get_player_stats, axis=1)
df_v3 = pd.concat([df_v3, player_features], axis=1)
print("✅ 선수 통계 피처 추가 완료")


🏃 선수 통계 피처 생성 중...
✅ 선수 통계 피처 추가 완료


### 6.2 팀 통계 피처

In [48]:
def get_team_stats(row):
    """해당 경기의 팀 통계 가져오기"""
    game_id = row['game_id']
    team_id = row['team_id']
    
    stats = team_stats_by_game.get(game_id)
    
    if stats is None or team_id not in stats:
        return pd.Series({
            'team_avg_pass_distance': global_team_avg['pass_distance'],
            'team_attack_style': global_team_avg['is_wide']
        })
    
    team_stat = stats[team_id]
    return pd.Series({
        'team_avg_pass_distance': team_stat.get(('pass_distance', 'mean'), global_team_avg['pass_distance']),
        'team_attack_style': team_stat.get(('is_wide', 'mean'), global_team_avg['is_wide'])
    })

print("\n⚽ 팀 통계 피처 생성 중...")
team_features = df_v3.apply(get_team_stats, axis=1)
df_v3 = pd.concat([df_v3, team_features], axis=1)
print("✅ 팀 통계 피처 추가 완료")


⚽ 팀 통계 피처 생성 중...
✅ 팀 통계 피처 추가 완료


### 6.3 경기 흐름 피처

In [49]:
print("\n⏱️ 경기 흐름 피처 생성 중...")

# match_info에서 득점 정보 가져오기
score_info = match_info[['game_id', 'home_score', 'away_score']].copy()

df_v3 = df_v3.merge(score_info, on='game_id', how='left')

# 득점차 계산 (내 팀 - 상대 팀)
df_v3['score_diff'] = np.where(
    df_v3['is_home'],
    df_v3['home_score'] - df_v3['away_score'],
    df_v3['away_score'] - df_v3['home_score']
)

# 경기 진행률 (0~1)
df_v3['match_period_normalized'] = df_v3['time_seconds'] / 5400  # 90분 = 5400초

# 후반 75분 이후 (4050초 이후)
df_v3['is_late_game'] = (df_v3['time_seconds'] >= 4050).astype(int)

print("✅ 경기 흐름 피처 추가 완료")


⏱️ 경기 흐름 피처 생성 중...
✅ 경기 흐름 피처 추가 완료


## 7. 최종 검증 및 저장

In [50]:
print("\n" + "=" * 50)
print("최종 검증")
print("=" * 50)

# 결측치 확인
print(f"\n📊 결측치 확인:")
missing = df_v3.isnull().sum()
missing = missing[missing > 0]
if len(missing) > 0:
    print(missing)
else:
    print("  ✅ 결측치 없음!")


최종 검증

📊 결측치 확인:
  ✅ 결측치 없음!


In [51]:
# 극단값 확인
print(f"\n📊 Phase 4 피처 범위:")
phase4_cols = [
    'player_avg_pass_distance', 'player_forward_ratio', 
    'player_success_rate', 'player_pass_count',
    'team_avg_pass_distance', 'team_attack_style',
    'score_diff', 'match_period_normalized', 'is_late_game'
]

for col in phase4_cols:
    print(f"  {col:30s}: [{df_v3[col].min():.2f}, {df_v3[col].max():.2f}]")


📊 Phase 4 피처 범위:
  player_avg_pass_distance      : [16.84, 16.84]
  player_forward_ratio          : [0.50, 0.50]
  player_success_rate           : [0.86, 0.86]
  player_pass_count             : [10.00, 10.00]
  team_avg_pass_distance        : [16.84, 16.84]
  team_attack_style             : [0.60, 0.60]
  score_diff                    : [-6.00, 6.00]
  match_period_normalized       : [0.00, 0.67]
  is_late_game                  : [0.00, 0.00]


In [52]:
# 최종 컬럼 수
print(f"\n✅ 최종 피처 개수: {len(df_v3.columns)}개")
print(f"  - Phase 3: 43개")
print(f"  - Phase 4 추가: 9개")
print(f"  - 임시 컬럼: {len(df_v3.columns) - 43 - 9}개")

# 불필요한 컬럼 제거
cols_to_drop = ['game_id', 'team_id', 'player_id', 'is_home', 'home_score', 'away_score']
df_v4 = df_v3.drop(columns=cols_to_drop)

print(f"\n✅ 정리 후 컬럼: {len(df_v4.columns)}개")


✅ 최종 피처 개수: 57개
  - Phase 3: 43개
  - Phase 4 추가: 9개
  - 임시 컬럼: 5개

✅ 정리 후 컬럼: 51개


In [53]:
# 저장
output_path = PROCESSED_DIR / 'train_final_passes_v4.csv'
df_v4.to_csv(output_path, index=False)
print(f"\n💾 저장 완료: {output_path}")
print(f"   Shape: {df_v4.shape}")

print("\n" + "=" * 50)
print("Phase 4 피처 생성 완료! 🎉")
print("=" * 50)


💾 저장 완료: ..\data\processed\train_final_passes_v4.csv
   Shape: (15435, 51)

Phase 4 피처 생성 완료! 🎉


## 8. Phase 4 데이터 검증 및 문제 진단

In [54]:
# 생성된 데이터 load
df_v4 = pd.read_csv(PROCESSED_DIR / 'train_final_passes_v4.csv')

### 8.1 기본 정보

In [55]:
print(f"\n✅ Shape: {df_v4.shape}")
print(f"✅ 컬럼 수: {len(df_v4.columns)}")


✅ Shape: (15435, 51)
✅ 컬럼 수: 51


### 8.2 Phase 4 피처 상세 분석

In [56]:
phase4_cols = [
    'player_avg_pass_distance', 'player_forward_ratio', 
    'player_success_rate', 'player_pass_count',
    'team_avg_pass_distance', 'team_attack_style',
    'score_diff', 'match_period_normalized', 'is_late_game'
]

print("\n" + "=" * 60)
print("Phase 4 피처 상세 분석")
print("=" * 60)

for col in phase4_cols:
    print(f"\n📊 {col}")
    print(f"   Min:    {df_v4[col].min():.4f}")
    print(f"   Max:    {df_v4[col].max():.4f}")
    print(f"   Mean:   {df_v4[col].mean():.4f}")
    print(f"   Std:    {df_v4[col].std():.4f}")
    print(f"   Unique: {df_v4[col].nunique():,}개")
    
    # 🚨 경고: 표준편차가 0이면 문제!
    if df_v4[col].std() < 0.01:
        print(f"   ⚠️  WARNING: 거의 변화 없음! (Std < 0.01)")


Phase 4 피처 상세 분석

📊 player_avg_pass_distance
   Min:    16.8420
   Max:    16.8420
   Mean:   16.8420
   Std:    0.0000
   Unique: 1개
   ⚠️  WARNING: 거의 변화 없음! (Std < 0.01)

📊 player_forward_ratio
   Min:    0.4992
   Max:    0.4992
   Mean:   0.4992
   Std:    0.0000
   Unique: 1개
   ⚠️  WARNING: 거의 변화 없음! (Std < 0.01)

📊 player_success_rate
   Min:    0.8634
   Max:    0.8634
   Mean:   0.8634
   Std:    0.0000
   Unique: 1개
   ⚠️  WARNING: 거의 변화 없음! (Std < 0.01)

📊 player_pass_count
   Min:    10.0000
   Max:    10.0000
   Mean:   10.0000
   Std:    0.0000
   Unique: 1개
   ⚠️  WARNING: 거의 변화 없음! (Std < 0.01)

📊 team_avg_pass_distance
   Min:    16.8420
   Max:    16.8420
   Mean:   16.8420
   Std:    0.0000
   Unique: 1개
   ⚠️  WARNING: 거의 변화 없음! (Std < 0.01)

📊 team_attack_style
   Min:    0.6047
   Max:    0.6047
   Mean:   0.6047
   Std:    0.0000
   Unique: 1개
   ⚠️  WARNING: 거의 변화 없음! (Std < 0.01)

📊 score_diff
   Min:    -6.0000
   Max:    6.0000
   Mean:   -0.0152
   Std:   

### 8.3 선수별 다양성 확인

In [ ]:
print("\n" + "=" * 60)
print("선수별 통계 다양성 확인")
print("=" * 60)

# 선수 통계 샘플 확인
print("\n📋 선수 통계 샘플 (10개):")
sample_cols = ['player_avg_pass_distance', 'player_forward_ratio', 
               'player_success_rate', 'player_pass_count']
print(df_v4[sample_cols].head(10))

# 고유값 확인
print("\n📊 고유값 개수:")
for col in sample_cols:
    print(f"   {col:30s}: {df_v4[col].nunique():,}개")


선수별 통계 다양성 확인

📋 선수 통계 샘플 (10개):
   player_avg_pass_distance  player_forward_ratio  player_success_rate  \
0                 16.841951              0.499166             0.863441   
1                 16.841951              0.499166             0.863441   
2                 16.841951              0.499166             0.863441   
3                 16.841951              0.499166             0.863441   
4                 16.841951              0.499166             0.863441   
5                 16.841951              0.499166             0.863441   
6                 16.841951              0.499166             0.863441   
7                 16.841951              0.499166             0.863441   
8                 16.841951              0.499166             0.863441   
9                 16.841951              0.499166             0.863441   

   player_pass_count  
0               10.0  
1               10.0  
2               10.0  
3               10.0  
4               10.0  
5            

In [65]:
# 🚨 문제 진단
print("\n" + "=" * 60)
print("문제 진단")
print("=" * 60)

issues = []


문제 진단


In [66]:
# 체크 1: 선수 통계가 모두 동일한가?
if df_v4['player_avg_pass_distance'].nunique() == 1:
    issues.append("❌ 선수별 평균 패스 거리가 모두 동일 (전체 평균만 사용됨)")
else:
    print("✅ 선수별 평균 패스 거리: 다양성 있음")

if df_v4['player_forward_ratio'].nunique() == 1:
    issues.append("❌ 선수별 전진 패스 비율이 모두 동일")
else:
    print("✅ 선수별 전진 패스 비율: 다양성 있음")

if df_v4['player_success_rate'].nunique() == 1:
    issues.append("❌ 선수별 성공률이 모두 동일")
else:
    print("✅ 선수별 성공률: 다양성 있음")

if df_v4['player_pass_count'].nunique() == 1:
    issues.append("❌ 선수별 패스 횟수가 모두 10 (기본값만 사용됨)")
else:
    print("✅ 선수별 패스 횟수: 다양성 있음")

In [67]:
# 체크 2: 팀 통계
if df_v4['team_avg_pass_distance'].nunique() == 1:
    issues.append("❌ 팀별 평균 패스 거리가 모두 동일")
else:
    print("✅ 팀별 평균 패스 거리: 다양성 있음")

if df_v4['team_attack_style'].nunique() == 1:
    issues.append("❌ 팀별 공격 스타일이 모두 동일")
else:
    print("✅ 팀별 공격 스타일: 다양성 있음")

In [68]:
# 체크 3: 경기 흐름 (이건 정상일 수 있음)
if df_v4['score_diff'].nunique() == 1:
    issues.append("⚠️  득점차가 모두 동일 (0-0 경기만?)")
else:
    print(f"✅ 득점차: {df_v4['score_diff'].nunique()}개 고유값")

if df_v4['is_late_game'].nunique() == 1 and df_v4['is_late_game'].iloc[0] == 0:
    print("⚠️  후반 75분 이후 데이터 없음 (정상일 수 있음)")
else:
    print(f"✅ 후반전 데이터: 있음")

✅ 득점차: 13개 고유값
⚠️  후반 75분 이후 데이터 없음 (정상일 수 있음)


### 8.4  최종 판정

In [69]:
print("\n" + "=" * 60)
print("최종 판정")
print("=" * 60)

if len(issues) > 0:
    print("\n🚨 문제 발견:")
    for issue in issues:
        print(f"   {issue}")
    print("\n→ Phase 4 피처가 제대로 생성되지 않았습니다!")
    print("→ 코드 수정이 필요합니다.")
else:
    print("\n✅ 모든 검증 통과!")
    print("→ Phase 4 피처가 정상적으로 생성되었습니다.")


최종 판정

🚨 문제 발견:
   ❌ 선수별 평균 패스 거리가 모두 동일 (전체 평균만 사용됨)
   ❌ 선수별 전진 패스 비율이 모두 동일
   ❌ 선수별 성공률이 모두 동일
   ❌ 선수별 패스 횟수가 모두 10 (기본값만 사용됨)
   ❌ 팀별 평균 패스 거리가 모두 동일
   ❌ 팀별 공격 스타일이 모두 동일

→ Phase 4 피처가 제대로 생성되지 않았습니다!
→ 코드 수정이 필요합니다.


### 8.5 추가 진단 정보

In [70]:
print("\n" + "=" * 60)
print("추가 진단 정보")
print("=" * 60)

# game_episode 확인
print(f"\n📋 game_episode 샘플:")
print(df_v4['game_episode'].head(5).tolist())

print(f"\n📊 고유 game_episode 수: {df_v4['game_episode'].nunique():,}개")
print(f"   (총 샘플: {len(df_v4):,}개)")


추가 진단 정보

📋 game_episode 샘플:
['126283_1', '126283_10', '126283_11', '126283_12', '126283_13']

📊 고유 game_episode 수: 15,435개
   (총 샘플: 15,435개)


In [71]:
# 상관관계 확인 (선수 통계끼리)
print(f"\n📊 선수 통계 간 상관관계:")
corr = df_v4[sample_cols].corr()
print(corr)

print("\n💡 해석:")
if corr.iloc[0, 1] > 0.99:  # 첫 번째와 두 번째 피처 상관관계
    print("   ⚠️  상관관계가 거의 1 → 모든 값이 동일할 가능성")
else:
    print("   ✅ 상관관계 정상 범위")


📊 선수 통계 간 상관관계:
                          player_avg_pass_distance  player_forward_ratio  \
player_avg_pass_distance                       NaN                   NaN   
player_forward_ratio                           NaN                   NaN   
player_success_rate                            NaN                   NaN   
player_pass_count                              NaN                   NaN   

                          player_success_rate  player_pass_count  
player_avg_pass_distance                  NaN                NaN  
player_forward_ratio                      NaN                NaN  
player_success_rate                       NaN                NaN  
player_pass_count                         NaN                NaN  

💡 해석:
   ✅ 상관관계 정상 범위
